# Training an Overcooked Policy using Madrona

In this notebook, we train an AI in Overcooked using the Multi-Agent PPO algorithm. The full training process can be completed in 2 minutes using our GPU-accelerated implementation of Overcooked, while the original implementation takes about an hour on the same hardware.

##Setup
Change the runtime type to use a T4 GPU (free colab tier). Run the next 3 cells to install dependencies for the notebook (should take about 1 minute in total).

In [ ]:
%pip uninstall -y build

!git clone -b overcooked_prebuilt --single-branch https://github.com/bsarkar321/madrona_rl_envs
%cd madrona_rl_envs

In [ ]:
!ln -s /content/madrona_rl_envs/cuda_so_12/libcudart.so.12 /usr/local/cuda/lib64/libcudart.so.12
!ln -s /content/madrona_rl_envs/cuda_so_12/libnvrtc.so.12 /usr/local/cuda/lib64/libnvrtc.so.12
!ln -s /content/madrona_rl_envs/cuda_so_12/libnvJitLink.so.12 /usr/local/cuda/lib64/libnvJitLink.so.12

In [ ]:
%pip install -e .
%pip install -e PantheonRL/overcookedgym/human_aware_rl/overcooked_ai

## Training

**Restart runtime here!** Then continue running the next blocks.

In this section, we will train the AI from scratch using MAPPO. The top score any agent can achieve in the "simple" layout is 234. For the set seed of 1, the average self-play score is 230.655 when the policy is non-deterministic, but it gets a score of 234 when it deterministically chooses an action.

Note that different seeds may result in different converged values, typically between 197 and 234. However, there are some cases when the score for the deterministic policy is very low, which occurs when the policy is in between different locally optimal conventions at the end of training.

In [ ]:
%cd /content/madrona_rl_envs

In [ ]:
%env MADRONA_MWGPU_KERNEL_CACHE=/content/madrona_rl_envs/caches/madrona_over_cache
import os
from train.MAPPO.main_player import MainPlayer

from train.config import get_config
from pathlib import Path

from train.env_utils import generate_env

from train.partner_agents import CentralizedAgent

import torch
import time
import numpy as np
import random

In [ ]:
args = get_config().parse_args("")
args.num_env_steps = 8000000
args.pop_size = 1
args.episode_length = 200
args.env_length = 200
args.env_name = "overcooked"
args.seed = 1
args.over_layout = "simple"
args.run_dir = "sp"
args.restored = 0
args.cuda = True

args.n_rollout_threads = 800
args.ppo_epoch = 7
args.layer_N = 1
args.hidden_size = 64
args.lr = 1e-2
args.critic_lr = 1e-2
args.entropy_coef = 0.0
args.linear_lr_decay = False

args.use_baseline = False

torch.manual_seed(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)

In [ ]:
device = 'cuda' if torch.cuda.is_available() and args.cuda else 'cpu'
print(device)

envs = generate_env(args.env_name, args.n_rollout_threads, args.over_layout, use_env_cpu=(device=='cpu'), use_baseline=args.use_baseline)

args.hanabi_name = args.over_layout if args.env_name == 'overcooked' else args.env_name

run_dir = (
        "train/"
        + args.hanabi_name
        + "/results/"
        + (args.run_dir)
        + "/"
        + str(args.seed)
    )
os.makedirs(run_dir, exist_ok=True)
with open(run_dir + "/" + "args.txt", "w", encoding="UTF-8") as file:
    file.write(str(args))
config = {
    'all_args': args,
    'envs': envs,
    'device': device,
    'num_agents': 2,
    'run_dir': Path(run_dir)
}

In [ ]:
# If you want to rerun this, remember to also run the two cells above so the environment is regenerated
# If you encounter an out of memory error, restart the runtime and start from the cell after "Restart runtime here!" above.
start = time.time()
ego = MainPlayer(config)
partner = CentralizedAgent(ego, 1)
envs.add_partner_agent(partner)
ego.run()
end = time.time()
print(f"Total time taken: {end - start} seconds")

# Testing the Trained Policy

By default, we run the "deterministic" version of the policy, which results in a score of 234, the highest possible value in the simple environment.

In [ ]:
import torch.nn as nn
from torch.distributions.categorical import Categorical


class Policy(nn.Module):

    def __init__(self, actor):
        super(Policy, self).__init__()

        self.base = actor.base.cnn.cnn
        self.act_layer = actor.act

    def forward(self, x: torch.Tensor):
        x = x.to(dtype=torch.float)
        x = self.base(x.permute((0, 3, 1, 2)))
        x = self.act_layer(x, deterministic=True)
        return x[0]

In [ ]:
run_dir = Path(run_dir)
args.model_dir = str(run_dir / 'models')

config = {
    'all_args': args,
    'envs': envs,
    'device': device,
    'num_agents': 2,
    'run_dir': run_dir
}

ego = MainPlayer(config)
ego.restore()
torch_network = Policy(ego.policy.actor)

actions = torch.zeros((2, args.n_rollout_threads, 1), dtype=int, device=device)

state1, state2 = envs.n_reset()
scores = torch.zeros(args.n_rollout_threads, device=device)
for i in range(args.env_length):
    actions[0, :, :] = torch_network(state1.obs)
    actions[1, :, :] = torch_network(state2.obs)
    (state1, state2), reward, _, _ = envs.n_step(actions)
    scores += reward[0, :]
score_vals, counts = torch.unique(scores, return_counts=True)

# printing scores here
print({x.item() : y.item() for x, y in zip(score_vals, counts)})

## Viewing and Interacting with Trained Agents

The next 3 cells convert the policy you just trained into an tensorflowjs model, allowing you to visualize your trained agent on-device through the javascript version of Overcooked. The game runs for 40 seconds, so the best score we were able to observe is 120, corresponding to 6 dishes served.

If you would like to see AI agents playing in other layouts, change the args.over_layout in the cells above, and retrain the agent.

In [ ]:
import onnx
from onnx_tf.backend import prepare

class SimplePolicy(nn.Module):

    def __init__(self, actor):
        super(SimplePolicy, self).__init__()

        self.base = actor.base.cnn.cnn
        self.act_layer = actor.act.action_out.linear

    def forward(self, x: torch.Tensor):
        x = self.base(x.permute((0, 3, 1, 2)))
        x = self.act_layer(x)
        return nn.functional.softmax(x, dim=1)

args.n_rollout_threads = 1

s_envs = generate_env(args.env_name, args.n_rollout_threads, args.over_layout)

args.hanabi_name = args.over_layout if args.env_name == 'overcooked' else args.env_name

torch_network = SimplePolicy(ego.policy.actor)

vobs, _ = s_envs.n_reset()
obs = vobs.obs.to(dtype=torch.float)

print("*" * 20, " TORCH ", "*" * 20)

print(torch_network)

print(obs.shape)

print(torch_network(obs))

print("*" * 20, " ONNX ", "*" * 20)
onnx_model_path = str(run_dir / "models" / f"MAPPO_{args.over_layout}_agent.onnx")

input_name = 'input'  # 'ppo_agent/ppo2_model/Ob'

torch.onnx.export(torch_network,
                  obs,
                  onnx_model_path,
                  export_params=True,
                  input_names=[input_name],
                  output_names=['output'],
                  opset_version=10)

onnx_model = onnx.load(onnx_model_path)

print(onnx_model.graph.input[0])

onnx.checker.check_model(onnx_model)

print("*" * 20, " TF ", "*" * 20)
tf_rep = prepare(onnx_model)
tf_model_dir = str(run_dir / 'models' / f'MAPPO_{args.over_layout}_agent')
tf_rep.export_graph(tf_model_dir)

tfjs_model_dir = f"overcooked_demo/static/assets/MAPPO_{args.over_layout}_agent"
tfjs_convert_command = f"""tensorflowjs_converter
                 --input_format=tf_saved_model
                 --output_format=tfjs_graph_model
                 --signature_name=serving_default
                 --saved_model_tags=serve
                 "{tf_model_dir}"
                 "{tfjs_model_dir}"
                 """
tfjs_convert_command = " ".join(tfjs_convert_command.split())

# This should only take about a minute in total. If it runs for longer, you may have to restart the runtime.
# In this case, you would need to run all the cells after the one that says "Restart runtime here!" but you
# can skip the cell for actually training the policy (since it is already saved to disk).
os.system(tfjs_convert_command)

In [ ]:
# Only run this ONCE for a runtime instance

# code to display on colab from https://stackoverflow.com/a/61504116
%cd /content/madrona_rl_envs/overcooked_demo
get_ipython().system_raw('python3 -m http.server 8888 &')
%cd /content/madrona_rl_envs

In [ ]:
from IPython.display import Javascript

def show_port(port, height=900):
  display(Javascript("""
  (async ()=>{
    fm = document.createElement('iframe')
    fm.src = await google.colab.kernel.proxyPort(%s)
    fm.width = '95%%'
    fm.height = '%d'
    fm.frameBorder = 0
    document.body.append(fm)
  })();
  """ % (port, height) ))

from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8888)"))

# displays the game in-line; you may have to wait for over a minute after this
# line runs before it appears, since it is running through javascript on your
# device
show_port(8888)